## Model Evaluation for college scorecard - earnings prediction

This notebook will use several different supervised learning regression algorithms to model earnings after college using College Scorecard data. The models included for evaluation will be:

1. Linear Regression
1. Decision Tree
1. Random Forest

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
seed = 12345
#load the data

data = pd.read_pickle('clean_notna_data.pickle')
X = data['X']
y = data['y']
name = X['INSTNM']
X.drop('INSTNM', axis = 1, inplace = True)
data = X.copy()
data['Y'] = y

X.head()

,MAIN,NUMBRANCH,ADM_RATE,PCIP01,PCIP03,PCIP04,PCIP05,PCIP09,PCIP10,PCIP11,...,CCSIZSET_17.0,CCSIZSET_nan,PFTFAC_ISNA,INEXPFTE_ISNA,AVGFACSAL_ISNA,GRAD_DEBT_MDN_ISNA,C150_4_ISNA,ADM_RATE_ISNA,TUITION_ISNA,TUITION_OUT_ISNT_IN
0,1,1.0,0.5575,0.0426,0.0019,0.0155,0.0000,0.0000,0.031,0.0756,...,0,0,0,0,0,0,0,0,0,1
1,1,1.0,0.9117,0.0000,0.0000,0.0000,0.0006,0.0540,0.000,0.0276,...,0,0,0,0,0,0,0,0,0,1
2,1,1.0,0.9050,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0574,...,0,0,0,0,0,0,0,0,0,1
3,1,1.0,0.6931,0.0000,0.0000,0.0000,0.0000,0.0393,0.000,0.1253,...,0,0,0,0,0,0,0,0,0,1
4,1,1.0,0.8462,0.0000,0.0000,0.0000,0.0041,0.1192,0.000,0.0164,...,0,0,0,0,0,0,0,0,0,1


### Data preprocessing
We need to split the data into training and test sets. Also, we need to think about possibly centering/normalizing the data.

Normalizing may make sense at least for linear regression, so we can understand the features a little bit better

Also, we can think about using PCA (if only for visualization purposes)

In [2]:
# Split the data into training and test
# Need to figure out best way to split time series data
# Do we split based only on colleges (i.e. each college is either train or test)
# Do we split based on college and year (i.e. each data entry is either train or test)
# Another way to split?

train_test_index = {0:'',1:'',2:'',3:'',4:'',5:''}
for i in range(0,6):
    train_test_index[i] = {
        'train': X[X['YEAR'] <= i+2002].index,
        'test' : X[X['YEAR'] == i + 2003].index
    }
X.drop('YEAR', axis = 1, inplace = True)

In [3]:
from sklearn.model_selection import train_test_split
train_val_index = list()
train_val_dataset = list()
for i in range(len(train_test_index)):
    X_i = X.iloc[train_test_index[i]['train']]
    y_i = y.iloc[train_test_index[i]['train']]
    train_val_dataset.append(train_test_split(X_i,y_i,test_size = .2, random_state = seed))
    train_val_index.append((train_val_dataset[i][0].index,train_val_dataset[i][1].index))

### Linear Regression
First try: no feature scaling

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as m_s_e
from sklearn.metrics import mean_absolute_error as m_a_e

r2_array = {'train': list(), 'val': list()}
mse_array = {'train': list(), 'val': list()}
mae_array = {'train': list(), 'val': list()}

linreg_coeffs = np.zeros_like(X.columns)
for i in range(0,6):
    X_train = X.loc[train_test_index[i]['train']]
    y_train = y.iloc[train_test_index[i]['train']]
    X_test = X.loc[train_test_index[i]['test']]
    y_test = y.iloc[train_test_index[i]['test']]
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)

    r2_array['train'].append(lin_reg.score(X_train, y_train))
    mse_array['train'].append(m_s_e(y_train, lin_reg.predict(X_train)))
    mae_array['train'].append(m_a_e(y_train, lin_reg.predict(X_train)))
    
    r2_array['val'].append(lin_reg.score(X_test, y_test))
    mse_array['val'].append(m_s_e(y_test, lin_reg.predict(X_test)))
    mae_array['val'].append(m_a_e(y_test, lin_reg.predict(X_test)))
    
    linreg_coeffs += lin_reg.coef_
    
linreg_coeffs /= 6
print('R^2 of training set: ',np.mean(r2_array['train']))
print('MSE of training set: ',np.mean(mse_array['train']))
print('RMSE of training set: ', np.sqrt(np.mean(mse_array['train'])))
print('MAE of training set: ',np.mean(mae_array['train']))
print()
print('R^2 of validation set: ',np.mean(r2_array['val']))
print('MSE of validation set: ',np.mean(mse_array['val']))
print('RMSE of validation set: ', np.sqrt(np.mean(mse_array['val'])))
print('MAE of validation set: ',np.mean(mae_array['val']))
print()
print('Coefficients:')
sorted_inds = np.argsort(-(np.abs(linreg_coeffs)))[:20]
for i in sorted_inds:
    print(X.columns[i], ': ', linreg_coeffs[i])

R^2 of training set:  0.8354695882995418
MSE of training set:  13975015.948902065
RMSE of training set:  3738.3172616702914
MAE of training set:  2675.3443939881936

R^2 of validation set:  0.8171420557005532
MSE of validation set:  14893159.98199828
RMSE of validation set:  3859.165710616516
MAE of validation set:  2843.6943177185126

Coefficients:
PCIP14 :  212045.018870773
PCIP54 :  204349.27633699894
PCIP29 :  197155.64544230278
PCIP10 :  193362.7003692057
PCIP46 :  191251.54628136917
PCIP15 :  190445.83456037077
PCIP47 :  190054.77813170548
PCIP48 :  189296.36537719518
PCIP52 :  187028.0943320917
PCIP51 :  186942.04911731093
PCIP11 :  186751.7580013466
PCIP40 :  186633.7413034727
PCIP41 :  184761.0342700867
PCIP24 :  183182.7248339264
PCIP22 :  182827.5252990272
PCIP27 :  182712.77417633907
PCIP16 :  182671.52385840903
PCIP49 :  182616.41419035205
PCIP31 :  182248.37746913123
PCIP43 :  181264.37969528104


Second try: feature scaling

In [5]:
X_scaled = scaler.fit_transform(X)

NameError: name 'scaler' is not defined

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

r2_array = {'train': list(), 'val': list()}
mse_array = {'train': list(), 'val': list()}
mae_array = {'train': list(), 'val': list()}

linreg_coeffs = np.zeros_like(X.columns)

accuracy_array = {'train': list(), 'val': list()}
for i in range(0,6):
    X_train = X_scaled[train_test_index[i]['train']]
    y_train = y[train_test_index[i]['train']]
    X_test = X_scaled[train_test_index[i]['test']]
    y_test = y[train_test_index[i]['test']]
    lin_reg = LinearRegression()
    lin_reg.fit(X_train, y_train)
    
    r2_array['train'].append(lin_reg.score(X_train, y_train))
    mse_array['train'].append(m_s_e(y_train, lin_reg.predict(X_train)))
    mae_array['train'].append(m_a_e(y_train, lin_reg.predict(X_train)))
    
    r2_array['val'].append(lin_reg.score(X_test, y_test))
    mse_array['val'].append(m_s_e(y_test, lin_reg.predict(X_test)))
    mae_array['val'].append(m_a_e(y_test, lin_reg.predict(X_test)))
    
    linreg_coeffs += lin_reg.coef_
    
linreg_coeffs /= 6
print('R^2 of training set: ',np.mean(r2_array['train']))
print('MSE of training set: ',np.mean(mse_array['train']))
print('RMSE of training set: ', np.sqrt(np.mean(mse_array['train'])))
print('MAE of training set: ',np.mean(mae_array['train']))
print()
print('R^2 of validation set: ',np.mean(r2_array['val']))
print('MSE of validation set: ',np.mean(mse_array['val']))
print('RMSE of validation set: ', np.sqrt(np.mean(mse_array['val'])))
print('MAE of validation set: ',np.mean(mae_array['val']))
print()
print('Coefficients:')
sorted_inds = np.argsort(-(np.abs(linreg_coeffs)))[:10]
for i in sorted_inds:
    print(X.columns[i], ': ', linreg_coeffs[i])

### Random forest


In [ ]:
#First using out-of-box parameters (which will probably lead to overfitting)
#use mse and then use mae
from sklearn.ensemble import RandomForestRegressor

oob_array = list()
r2_array = {'train': list(), 'val': list()}
mse_array = {'train': list(), 'val': list()}
mae_array = {'train': list(), 'val': list()}

feature_importances = np.zeros_like(X.columns)
for i in range(0,6):
    X_train = X.loc[train_val_index[i][0]]
    y_train = y.iloc[train_val_index[i][0]]
    X_test = X.loc[train_val_index[i][1]]
    y_test = y.iloc[train_val_index[i][1]]
    rand_forest = RandomForestRegressor(criterion = 'mse',oob_score = True, random_state = seed,
                                       n_estimators = 100)
    rand_forest.fit(X_train, y_train)
    
    oob_array.append(rand_forest.oob_score_)
    
    r2_array['train'].append(rand_forest.score(X_train, y_train))
    mse_array['train'].append(m_s_e(y_train, rand_forest.predict(X_train)))
    mae_array['train'].append(m_a_e(y_train, rand_forest.predict(X_train)))
    
    r2_array['val'].append(rand_forest.score(X_test, y_test))
    mse_array['val'].append(m_s_e(y_test, rand_forest.predict(X_test)))
    mae_array['val'].append(m_a_e(y_test, rand_forest.predict(X_test)))
    
    
    
    feature_importances += rand_forest.feature_importances_
feature_importances /= 6

print('OOB score : ', np.mean(oob_array))
print('R^2 of training set: ',np.mean(r2_array['train']))
print('MSE of training set: ',np.mean(mse_array['train']))
print('RMSE of training set: ', np.sqrt(np.mean(mse_array['train'])))
print('MAE of training set: ',np.mean(mae_array['train']))
print()
print('R^2 of validation set: ',np.mean(r2_array['val']))
print('MSE of validation set: ',np.mean(mse_array['val']))
print('RMSE of validation set: ', np.sqrt(np.mean(mse_array['val'])))
print('MAE of validation set: ',np.mean(mae_array['val']))
print()



In [ ]:
x = np.arange(20)
sorted_inds = np.argsort(-(feature_importances))[:20]
sorted_colnames = X.columns[sorted_inds]
fig = plt.figure(figsize = (15,5))
plt.bar(x,feature_importances[sorted_inds], width = 0.8)
plt.xticks(x,(sorted_colnames))
plt.show()
for i in range(0,20):
    print(sorted_colnames[i],': ', feature_importances[sorted_inds[i]])

In [ ]:
accuracy_dict = dict()

In [ ]:
n_estimators = 120
max_features = [0.3,0.4,0.5,0.66,0.75,0.9,0.99, 1]

# initialize accuracy dict which will have accuracy values for each run of the parameter tuning

for max_f in max_features:
    accuracy_dict[max_f] = {'r2':
                                {'train':
                                    {'list':list(),
                                     'mean': 0},
                                'test':
                                    {'list':list(),
                                     'mean': 0},
                                'oob':
                                    {'list':list(),
                                     'mean': 0}},
                            'mse':
                                {'train':
                                    {'list':list(),
                                     'mean': 0},
                                'test':
                                    {'list':list(),
                                     'mean': 0},
                                'oob':
                                    {'list':list(),
                                     'mean': 0}},
                            'mae':
                                {'train':
                                    {'list':list(),
                                     'mean': 0},
                                'test':
                                    {'list':list(),
                                     'mean': 0},
                                'oob':
                                    {'list':list(),
                                     'mean': 0}}}

for max_f in max_features:
    for i in range(0,6):
        X_train = X.loc[train_test_index[i]['train']]
        y_train = y.iloc[train_test_index[i]['train']]
        X_test = X.loc[train_test_index[i]['test']]
        y_test = y.iloc[train_test_index[i]['test']]
        rand_forest = RandomForestRegressor(n_estimators = n_estimators, max_features = max_f,
                                        criterion = 'mse', oob_score = True, random_state = seed)
        rand_forest.fit(X_train, y_train)

        accuracy_dict[max_f]['r2']['oob']['list'].append(rand_forest.oob_score_)
        accuracy_dict[max_f]['mse']['oob']['list'].append(m_s_e(y_train, rand_forest.oob_prediction_))
        accuracy_dict[max_f]['mae']['oob']['list'].append(m_a_e(y_train, rand_forest.oob_prediction_))
        
        accuracy_dict[max_f]['r2']['train']['list'].append(rand_forest.score(X_train, y_train))
        accuracy_dict[max_f]['mse']['train']['list'].append(m_s_e(y_train, rand_forest.predict(X_train)))
        accuracy_dict[max_f]['mae']['train']['list'].append(m_a_e(y_train, rand_forest.predict(X_train)))
        
        accuracy_dict[max_f]['r2']['test']['list'].append(rand_forest.score(X_test, y_test))
        accuracy_dict[max_f]['mse']['test']['list'].append(m_s_e(y_test, rand_forest.predict(X_test)))
        accuracy_dict[max_f]['mae']['test']['list'].append(m_a_e(y_test, rand_forest.predict(X_test)))
    
    
    accuracy_dict[max_f]['r2']['oob']['mean'] = np.mean(accuracy_dict[max_f]['r2']['oob']['list'])
    accuracy_dict[max_f]['mse']['oob']['mean'] = np.mean(accuracy_dict[max_f]['mse']['oob']['list'])
    accuracy_dict[max_f]['mae']['oob']['mean'] = np.mean(accuracy_dict[max_f]['mae']['oob']['list'])
    
    accuracy_dict[max_f]['r2']['train']['mean'] = np.mean(accuracy_dict[max_f]['r2']['train']['list'])
    accuracy_dict[max_f]['mse']['train']['mean'] = np.mean(accuracy_dict[max_f]['mse']['train']['list'])
    accuracy_dict[max_f]['mae']['train']['mean'] = np.mean(accuracy_dict[max_f]['mae']['train']['list'])
                            
    accuracy_dict[max_f]['r2']['test']['mean'] = np.mean(accuracy_dict[max_f]['r2']['test']['list'])
    accuracy_dict[max_f]['mse']['test']['mean'] = np.mean(accuracy_dict[max_f]['mse']['test']['list'])
    accuracy_dict[max_f]['mae']['test']['mean'] = np.mean(accuracy_dict[max_f]['mae']['test']['list'])
    print(max_f)
                            
#change other parameters as we see fit (probably max_depth, min_size_leaf, min_size_split)

In [ ]:
column_pairs = list()
groups = ['oob', 'train', 'test']
metrics = ['r2', 'rmse', 'mae']
for group in groups:
    for metric in metrics:
        column_pairs.append((group, metric))

micolumns = pd.MultiIndex.from_tuples(column_pairs)

accuracy_means = pd.DataFrame(columns = micolumns)

In [ ]:
for max_f in max_features:
    oob_r2 = accuracy_dict[max_f]['r2']['oob']['mean']
    oob_mse = accuracy_dict[max_f]['mse']['oob']['mean']
    oob_rmse = np.sqrt(oob_mse)
    oob_mae = accuracy_dict[max_f]['mae']['oob']['mean']
    
    train_r2 = accuracy_dict[max_f]['r2']['train']['mean']
    train_mse = accuracy_dict[max_f]['mse']['train']['mean']
    train_rmse = np.sqrt(train_mse)
    train_mae = accuracy_dict[max_f]['mae']['train']['mean']
    
    test_r2 = accuracy_dict[max_f]['r2']['test']['mean']
    test_mse = accuracy_dict[max_f]['mse']['test']['mean']
    test_rmse = np.sqrt(test_mse)
    test_mae = accuracy_dict[max_f]['mae']['test']['mean']
    
    accuracy_means.loc[max_f] = [oob_r2, oob_rmse, oob_mae,train_r2,train_rmse,train_mae,
                                 test_r2,test_rmse,test_mae]
accuracy_means.sort_index(inplace = True)
accuracy_means

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize =(16,4))
ax1.plot(accuracy_means.index, accuracy_means['oob', 'r2'])
ax1.title.set_text('OOB - R^2')
ax1.set_xlabel('max features')
ax2.plot(accuracy_means.index, accuracy_means['oob', 'rmse'])
ax2.title.set_text('OOB -  Square Root MSE')
ax2.set_xlabel('max features')
ax3.plot(accuracy_means.index, accuracy_means['oob', 'mae'])
ax3.title.set_text('OOB - MAE')
ax3.set_xlabel('max features')
plt.show()

##### Using optimal max_features to find feature importance

In [ ]:
best_max_f = 0.5

r2_array = {'train': list(), 'test': list()}
mse_array = {'train': list(), 'test': list()}
mae_array = {'train': list(), 'test': list()}

feature_importances = np.zeros_like(X.columns)
for i in range(0,6):
    X_train = X.loc[train_test_index[i]['train']]
    y_train = y.iloc[train_test_index[i]['train']]
    X_test = X.loc[train_test_index[i]['test']]
    y_test = y.iloc[train_test_index[i]['test']]
    rand_forest = RandomForestRegressor(criterion = 'mse',oob_score = True, max_features = best_max_f,
                                        random_state = seed, n_estimators = 100)
    rand_forest.fit(X_train, y_train)
    
    oob_array.append(rand_forest.oob_score_)
    
    r2_array['train'].append(rand_forest.score(X_train, y_train))
    mse_array['train'].append(m_s_e(y_train, rand_forest.predict(X_train)))
    mae_array['train'].append(m_a_e(y_train, rand_forest.predict(X_train)))
    
    r2_array['test'].append(rand_forest.score(X_test, y_test))
    mse_array['test'].append(m_s_e(y_test, rand_forest.predict(X_test)))
    mae_array['test'].append(m_a_e(y_test, rand_forest.predict(X_test)))
    
    feature_importances += rand_forest.feature_importances_
feature_importances /= 6

print('OOB score : ', np.mean(oob_array))
print('R^2 of training set: ',np.mean(r2_array['train']))
print('MSE of training set: ',np.mean(mse_array['train']))
print('RMSE of training set: ', np.sqrt(np.mean(mse_array['train'])))
print('MAE of training set: ',np.mean(mae_array['train']))
print()
print('R^2 of Test set: ',np.mean(r2_array['test']))
print('MSE of Test set: ',np.mean(mse_array['test']))
print('RMSE of Test set: ', np.sqrt(np.mean(mse_array['test'])))
print('MAE of Test set: ',np.mean(mae_array['test']))
print()

In [ ]:
x = np.arange(20)
sorted_inds = np.argsort(-(feature_importances))[:20]
sorted_colnames = X.columns[sorted_inds]
fig = plt.figure(figsize = (15,5))
plt.bar(x,feature_importances[sorted_inds], width = 0.8)
plt.xticks(x,(sorted_colnames))
plt.show()
for i in range(0,20):
    print(sorted_colnames[i],': ', feature_importances[sorted_inds[i]])

Least accurate predictions

In [ ]:
prediction = rand_forest.predict(X_test)
diff = y_test - prediction
absdiff = np.abs(prediction - y_test)
sorted_inds = np.argsort(-(absdiff))[:10]

for i in sorted_inds:
    name_index = train_test_index[5]['test'][i]
    print(name[name_index])
    print('predicted value:\t',prediction[i])
    print('real value:\t\t', y_test[name_index])
    print('difference:\t\t', diff[name_index])
    print()

Most accurate predictions

In [ ]:
prediction = rand_forest.predict(X_test)
diff = y_test - prediction
absdiff = np.abs(prediction - y_test)
sorted_inds = np.argsort(absdiff)[:10]

for i in sorted_inds:
    name_index = train_test_index[5]['test'][i]
    print(name[name_index])
    print(prediction[i])
    print(y_test[name_index])
    print(diff[name_index])
    print()